In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Chronic_obstructive_pulmonary_disease_(COPD)"
cohort = "GSE84046"

# Input paths
in_trait_dir = "../../input/GEO/Chronic_obstructive_pulmonary_disease_(COPD)"
in_cohort_dir = "../../input/GEO/Chronic_obstructive_pulmonary_disease_(COPD)/GSE84046"

# Output paths
out_data_file = "../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/GSE84046.csv"
out_gene_data_file = "../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/gene_data/GSE84046.csv"
out_clinical_data_file = "../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/clinical_data/GSE84046.csv"
json_path = "../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"The impact of protein quantity during energy restriction on genome-wide gene expression analysis in human adipose tissue"
!Series_summary	"Overweight is a growing health problem worldwide. The most effective strategy to reduce weight is energy restriction (ER): restriction of food intake without malnutrition. ER has been shown to be beneficial in disease prevention, healthy aging, and inflammation. Recent studies suggest that reducing the protein content of a diet contributes to the beneficial effects by ER. The first objective of our study was to assess the effect of energy restriction on changes in gene expression in adipose tissue. Secondly, the changes in gene expression were compared between a high protein diet and a normal protein diet during energy restriction. In a parallel double-blinded study, overweight older subjects adhered to a 25% ER diet, either combined with high protein intake  (HP-ER, 1.7 g/kg per day), or with normal protein in

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Review data availability and determine preprocessing approach

# 1. Gene Expression Data Availability
# Based on the background information, this study conducts genome-wide gene 
# expression analysis on adipose tissue
is_gene_available = True  # This is gene expression data, not just miRNA or methylation

# 2. Variable Availability and Data Type Conversion

# 2.1 Trait (COPD) Availability
# This study is about protein content in diet during energy restriction
# Not related to COPD, so trait data is not available
trait_row = None

# 2.2 Gender Availability
# Gender is available in row 4, labeled as 'sexe'
gender_row = 4

def convert_gender(value):
    if value is None:
        return None
    if ':' in value:
        value = value.split(': ')[1].strip()
    if value.lower() == 'female':
        return 0
    elif value.lower() == 'male':
        return 1
    return None

# 2.3 Age Availability
# Age is not directly given but can be inferred from date of birth in row 5
age_row = 5

def convert_age(value):
    if value is None:
        return None
    if ':' in value:
        value = value.split(': ')[1].strip()
    
    # Extract birth year from format "date of birth (dd-mm-yyyy): YYYY-MM-DD"
    if '-' in value:
        try:
            birth_year = int(value.split('-')[0])
            # Assuming study was conducted around 2014 (based on context)
            # Most people in study are from 1940s-1950s which would make them ~60-70 years old
            study_year = 2014
            age = study_year - birth_year
            return age
        except:
            return None
    return None

# No conversion function for trait since it's not available
def convert_trait(value):
    return None

# 3. Save Metadata - Initial filtering
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Skip clinical feature extraction since trait data is not available
# The function will return False since is_trait_available is False


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Chronic_obstructive_pulmonary_disease_(COPD)/GSE84046/GSE84046_series_matrix.txt.gz
Gene data shape: (33297, 44)
First 20 gene/probe identifiers:
Index(['7892501', '7892502', '7892503', '7892504', '7892505', '7892506',
       '7892507', '7892508', '7892509', '7892510', '7892511', '7892512',
       '7892513', '7892514', '7892515', '7892516', '7892517', '7892518',
       '7892519', '7892520'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The gene identifiers are microarray probe IDs, not human gene symbols
requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Let's look for platform information in the SOFT file to understand the annotation better
print("\nSearching for platform information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        if '!Series_platform_id' in line:
            print(line.strip())
            break
        if i > 100:  # Limit search to first 100 lines
            print("Platform ID not found in first 100 lines")
            break

# Check if the SOFT file includes any reference to gene symbols
print("\nSearching for gene symbol information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    gene_symbol_lines = []
    for i, line in enumerate(f):
        if 'GENE_SYMBOL' in line or 'gene_symbol' in line.lower() or 'symbol' in line.lower():
            gene_symbol_lines.append(line.strip())
        if i > 1000 and len(gene_symbol_lines) > 0:  # Limit search but ensure we found something
            break
    
    if gene_symbol_lines:
        print("Found references to gene symbols:")
        for line in gene_symbol_lines[:5]:  # Show just first 5 matches
            print(line)
    else:
        print("No explicit gene symbol references found in first 1000 lines")

# Look for alternative annotation files or references in the directory
print("\nChecking for additional annotation files in the directory:")
all_files = os.listdir(in_cohort_dir)
print([f for f in all_files if 'annotation' in f.lower() or 'platform' in f.lower() or 'gpl' in f.lower()])



Gene annotation preview:
Columns in gene annotation: ['ID', 'GB_LIST', 'SPOT_ID', 'seqname', 'RANGE_GB', 'RANGE_STRAND', 'RANGE_START', 'RANGE_STOP', 'total_probes', 'gene_assignment', 'mrna_assignment', 'category']
{'ID': ['7896736', '7896738', '7896740', '7896742', '7896744'], 'GB_LIST': [nan, nan, 'NM_001005240,NM_001004195,NM_001005484,BC136848,BC136907', 'BC118988,AL137655', 'NM_001005277,NM_001005221,NM_001005224,NM_001005504,BC137547'], 'SPOT_ID': ['chr1:53049-54936', 'chr1:63015-63887', 'chr1:69091-70008', 'chr1:334129-334296', 'chr1:367659-368597'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': [53049.0, 63015.0, 69091.0, 334129.0, 367659.0], 'RANGE_STOP': [54936.0, 63887.0, 70008.0, 334296.0, 368597.0], 'total_probes': [7.0, 31.0, 24.0, 6.0, 36.0], 'gene_assignment': ['---', '---', 'NM_001005240 // OR4F17 // olfactory 

No explicit gene symbol references found in first 1000 lines

Checking for additional annotation files in the directory:
[]


### Step 6: Gene Identifier Mapping

In [7]:
# Analyze the gene annotation dataframe to determine the mapping from probe IDs to gene symbols
# First, confirm the columns containing probe IDs 
print("First 5 IDs from gene expression data:", gene_data.index[:5].tolist())
print("First 5 IDs from gene annotation:", gene_annotation['ID'][:5].tolist())

# Looking at the gene_assignment column which contains gene symbols
gene_assignment_sample = gene_annotation['gene_assignment'].head(3).tolist()
print("\nSample gene_assignment entries:")
for entry in gene_assignment_sample:
    print(f"{entry[:200]}...")

# Extract gene mapping from annotation
# The ID column contains the probe identifiers
# The gene_assignment column contains gene symbols but needs parsing
print("\nCreating gene mapping dataframe...")
mapping_df = get_gene_mapping(gene_annotation, 'ID', 'gene_assignment')
print(f"Original mapping shape: {mapping_df.shape}")
print("Sample mapped entries:")
print(mapping_df.head(3))

# Apply the gene mapping to convert probe-level measurements to gene-level expression
print("\nApplying gene mapping to gene expression data...")
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Gene-level expression data shape: {gene_data.shape}")
print("First 10 gene symbols after mapping:")
print(list(gene_data.index[:10]))

# Preview the gene expression data
print("\nPreview of gene expression data:")
preview_sample = preview_df(gene_data, n=3)
preview_cols = list(preview_sample.keys())[:5]  # Show only first 5 columns to avoid clutter
preview_data = {col: preview_sample[col] for col in preview_cols}
print(preview_data)


First 5 IDs from gene expression data: ['7892501', '7892502', '7892503', '7892504', '7892505']
First 5 IDs from gene annotation: ['7896736', '7896738', '7896740', '7896742', '7896744']

Sample gene_assignment entries:
---...
---...
NM_001005240 // OR4F17 // olfactory receptor, family 4, subfamily F, member 17 // 19p13.3 // 81099 /// NM_001004195 // OR4F4 // olfactory receptor, family 4, subfamily F, member 4 // 15q26.3 // 26682 ...

Creating gene mapping dataframe...
Original mapping shape: (33297, 2)
Sample mapped entries:
        ID                                               Gene
0  7896736                                                ---
1  7896738                                                ---
2  7896740  NM_001005240 // OR4F17 // olfactory receptor, ...

Applying gene mapping to gene expression data...


Gene-level expression data shape: (56391, 44)
First 10 gene symbols after mapping:
['A-', 'A-2', 'A-52', 'A-E', 'A-I', 'A-II', 'A-IV', 'A-V', 'A0', 'A1']

Preview of gene expression data:
{'GSM2226420': [33.68976687768011, 1.9937525379999999, 3.3021436486666667], 'GSM2226421': [33.80631276691363, 1.8987550603333334, 3.3720884666666664], 'GSM2226422': [34.00824614293309, 1.929899198, 3.3661534399999997], 'GSM2226423': [33.61255679126494, 1.9025789763333334, 3.3930164499999997], 'GSM2226424': [33.662630774239126, 2.024719238, 3.3526896666666666]}


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")

# 2. Since we previously determined that trait data is not available (trait_row = None),
# we can't create clinical features or linked data for this cohort
print("\nNo COPD-related trait data available in this dataset.")

# 3. Create a minimal clinical dataframe with just the demographic information
# Get the sample IDs from gene data
sample_ids = normalized_gene_data.columns.tolist()

# Create clinical dataframe with gender and age if available
clinical_features_list = []

# Add gender information if available
if gender_row is not None:
    gender_series = clinical_data.iloc[gender_row].drop('!Sample_geo_accession', errors='ignore')
    gender_series = gender_series.map(convert_gender)
    gender_series.name = 'Gender'
    clinical_features_list.append(gender_series)

# Add age information if available
if age_row is not None:
    age_series = clinical_data.iloc[age_row].drop('!Sample_geo_accession', errors='ignore') 
    age_series = age_series.map(convert_age)
    age_series.name = 'Age'
    clinical_features_list.append(age_series)

# Create clinical dataframe if we have features
if clinical_features_list:
    clinical_df = pd.concat(clinical_features_list, axis=1)
    
    # Save clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data (demographic only) saved to {out_clinical_data_file}")
    
    print("\nClinical data preview:")
    print(clinical_df.head(3))
else:
    print("No usable clinical features found.")

# 4. Final validation - use is_final=False since this is an initial filter-out case
note = "This dataset contains gene expression from adipose tissue in a protein diet study, not related to COPD."
is_usable = validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=False  # No COPD trait data
)

print(f"\nDataset usability assessment: {'Usable' if is_usable else 'Not usable'} for COPD analysis")
print(f"Assessment details saved to {json_path}")

Normalized gene data shape: (20124, 44)


Gene expression data saved to ../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/gene_data/GSE84046.csv

No COPD-related trait data available in this dataset.
Clinical data (demographic only) saved to ../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/clinical_data/GSE84046.csv

Clinical data preview:
            Gender  Age
GSM2226420       1   62
GSM2226421       1   70
GSM2226422       1   62

Dataset usability assessment: Not usable for COPD analysis
Assessment details saved to ../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/cohort_info.json
